## Notebook []
#### 全国気象情報を収集

##### [pythonで気象データを取得](https://qiita.com/apiss/items/d25a3b92a92325dedb42)
##### [過去の気象データ検索](https://www.data.jma.go.jp/obd/stats/etrn/select/prefecture00.php?prec_no=&block_no=&year=&month=&day=&view=)
##### [47都道府県県庁所在地の緯度経度データ](https://techtech-sorae.com/jp_city_latlon/)

In [1]:
!pip install geopy
!pip install lxml
!pip install html5lib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 10.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 4.1 MB/s eta 0:00:00


In [2]:
import pandas as pd
import requests
import io
from bs4 import BeautifulSoup
from tqdm import tqdm
import re
from geopy.distance import geodesic
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta
import lxml
import html5lib

In [3]:
####################################################
# 都道府県ごとの府県コードを取得する
####################################################
URL = "https://www.soumu.go.jp/main_content/000608358.csv" # ここにcsvファイルのURLを書く

r = requests.get(URL)
df = pd.read_csv(io.BytesIO(r.content),sep=",", encoding="shiftjis")
#print(df)
df_temp = df[(df['sityouson-code']==0)][['ken-code','ken-name']]
df_temp.rename(columns={'ken-code': 'pref_code' , 'ken-name': 'pref_name'}, inplace=True)
df_temp.to_csv('~/datasets/ETL/pref_list.csv', header=True, index=False, encoding="utf8")

In [4]:
####################################################
# 都道府県ごとの県庁所在地・緯度経度を取得する
####################################################
URL = "https://techtech-sorae.com/wp-content/uploads/2021/07/pref_lat_lon.csv" # ここにcsvファイルのURLを書く
r = requests.get(URL)
df_temp = pd.read_csv(io.BytesIO(r.content), encoding="utf8")
#誤記修正
df_temp.replace({'pref_name': {'京都県':'京都府'}}, inplace=True)
df_temp.to_csv("~/datasets/ETL/pref_lat_lon.csv", index=False)

In [5]:
####################################################
# アメダスの観測地点の一覧を取得できるクラス
####################################################
class AmedasStationScraper(object):
    #----------------------------------------------
    # イニシャライザー
    #----------------------------------------------
    def __init__(self, encoding='utf-8'):
        self.encoding = encoding
        url = 'https://www.data.jma.go.jp/obd/stats/etrn/select/prefecture00.php?prec_no=&block_no=&year=&month=&day=&view='
        self.soup = self._get_soup(url, encoding=self.encoding)
    #----------------------------------------------
    #　HTMLパース
    #----------------------------------------------
    def _get_soup(self, url: str, encoding: str = None):
        html = requests.get(url)
        if encoding is not None:
            html.encoding = encoding
        soup = BeautifulSoup(html.text, "html.parser")
        return soup
    #----------------------------------------------
    # 地点一覧からリンク情報取得
    #----------------------------------------------
    def run(self):
        area_list, area_link_list = self.get_all_area_link()
        df = self.get_all_station_link(area_list, area_link_list)
        return df
    #----------------------------------------------
    # リンク情報からエリア情報を生成
    #----------------------------------------------
    def get_all_area_link(self):
        elements = self.soup.find_all('area')
        area_list = [element['alt'] for element in elements]
        area_link_list = [element['href'] for element in elements]
        return area_list, area_link_list
    #----------------------------------------------
    # 観測地点情報DF
    #----------------------------------------------
    def get_all_station_link(self, area_list, area_link_list) -> pd.DataFrame:
        dfs = list()
        for area, area_link in tqdm(zip(area_list, area_link_list), total = len(area_list)):
            dfs.append(self.get_station_data(area, area_link))
        df = pd.concat(dfs).reset_index(drop=True)
        return self.format_df(df)
    #----------------------------------------------
    #　都道府県ごとの観測一覧を取得
    #----------------------------------------------
    def get_station_data(self, area, area_link) -> pd.DataFrame:
        url = 'https://www.data.jma.go.jp/obd/stats/etrn/select/' + area_link
        soup = self._get_soup(url, encoding=self.encoding)
        elements = soup.find_all('area')
        station_list = [element['alt'] for element in elements]
        station_link_list = [element['href'].strip(
            '../') for element in elements]
        station_info = [element['onmouseover'] if element.has_attr(
            'onmouseover') else '-' for element in elements]
        assert len(station_list) == len(station_link_list)
        data = {'station': station_list,
                'url': station_link_list, 'info': station_info}
        df = pd.DataFrame(data)
        df['area'] = area
        return df[['area', 'station', 'url', 'info']]
    #----------------------------------------------
    # 都道府県ごとの観測地詳細情報を取得
    #----------------------------------------------
    def defaultfind(self, pattern, s, default=None, callback=None, ix=None):
        cont = re.findall(pattern, s)
        if len(cont) == 0:
            return default
        else:
            if callback is not None:
                return callback(pattern, s, ix)
            return cont[0]
    #----------------------------------------------
    # Javascriptパラメタを解析し詳細情報に転用する
    #----------------------------------------------
    def format_df(self, df):
        # prec_no
        df['prec_no'] = df.url.apply(
            lambda x: self.defaultfind("prec_no=\d{1,2}&", x))
        df = df.dropna()
        df.prec_no = df.prec_no.apply(lambda x: int(x[8:-1]))
        # block_no
        df['block_no'] = df.url.apply(
            lambda x: self.defaultfind("block_no=\d{4,6}&", x))
        df = df.dropna()
        df.block_no = df.block_no.apply(lambda x: x[9:-1])
        # block_no
        df['type'] = df['info'].apply(
            lambda x: self.defaultfind("Point\('.'", x))
        df = df.dropna()
        df['type'] = df['type'].apply(lambda x: x[7:-1])
        #緯度 latitude
        df['latitude'] = df['info'].apply(
            lambda x: self.defaultfind("'(.*?)'", x, "*", get_latitude, 4))
        #経度 longitude
        df['logtitude'] = df['info'].apply(
            lambda x: self.defaultfind("'(.*?)'", x, "*", get_logtitude, 6))
        
        # 不要部分削除
        df.drop(['url', 'info'], inplace=True, axis=1)
        df.drop_duplicates(inplace=True)
        return df.reset_index(drop=True)


####################################################
# 緯度経度情報：計算
####################################################
def get_latitude(pattern, s, ix):
    cont = re.findall(pattern, s)
    res = float(cont[ix]) + float(cont[ix+1]) / 60
    #print(f'LAT %s %s' % (cont[ix:ix+2], res))
    return res
####################################################
# 緯度経度情報：計算
####################################################
def get_logtitude(pattern, s, ix):
    cont = re.findall(pattern, s)
    res = float(cont[ix]) + float(cont[ix+1]) / 60
    #print(f'LAT %s %s' % (cont[ix:ix+2], s))
    return res

In [6]:
####################################################
# 観測地点情報作成
####################################################
dfSTATION = AmedasStationScraper().run()

100%|██████████| 61/61 [00:06<00:00,  8.85it/s]


In [7]:
####################################################
#都道府県コードを付与
####################################################
def pref_code(prec_no):
    dic = {
        11:  1 , #北海道
        12:  1 , #北海道
        13:  1 , #北海道
        14:  1 , #北海道
        15:  1 , #北海道
        16:  1 , #北海道
        17:  1 , #北海道
        18:  1 , #北海道
        19:  1 , #北海道
        20:  1 , #北海道
        21:  1 , #北海道
        22:  1 , #北海道
        23:  1 , #北海道
        24:  1 , #北海道
        31:  2 , #青森
        32:  5 , #秋田
        33:  3 , #岩手
        34:  4 , #宮城
        35:  6 , #山形
        36:  7 , #福島
        40:  8 , #茨城
        41:  9 , #栃木
        42: 10 , #群馬
        43: 11 , #埼玉
        44: 13 , #東京
        45: 12 , #千葉
        46: 14 , #神奈川
        48: 20 , #長野
        49: 19 , #山梨
        50: 22 , #静岡
        51: 23 , #愛知
        52: 21 , #岐阜
        53: 24 , #三重
        54: 15 , #新潟
        55: 16 , #富山
        56: 17 , #石川
        57: 18 , #福井
        60: 25 , #滋賀
        61: 26 , #京都
        62: 27 , #大阪
        63: 28 , #兵庫
        64: 29 , #奈良
        65: 30 , #和歌山
        66: 33 , #岡山
        67: 34 , #広島
        68: 32 , #島根
        69: 31 , #鳥取
        71: 36 , #徳島
        72: 36 , #徳島
        72: 37 , #香川
        73: 38 , #愛媛
        74: 39 , #高知
        81: 35 , #山口
        82: 40 , #福岡
        83: 44 , #大分
        84: 42 , #長崎
        85: 41 , #佐賀
        86: 43 , #熊本
        87: 45 , #宮崎
        88: 46 , #鹿児島
        91: 47 , #沖縄
        92: 47 , #沖縄
        93: 47 , #沖縄
        94: 47 , #沖縄
    }
    if prec_no in dic:
        result = dic[prec_no]
    else:
        result = 1
    return result

####################################################
#観測地点一覧に都道府県コードを付与
#stationAを除外する（日照時間など不足するため)
####################################################
#dfSTATION.info()
dfSTATION = dfSTATION[dfSTATION['prec_no'] != 99] #南極は除外する
dfSTATION = dfSTATION[dfSTATION['type'] != 'a'] #a は除外する
dfSTATION.rename(columns={'type': 'amedas_type'}, inplace=True)
dfSTATION['pref_code'] = dfSTATION['prec_no'].apply(
    lambda x: pref_code(x))

In [8]:
#デバッグ
#dfSTATION[dfSTATION['prec_no']==43]

In [9]:
####################################################
#都道府県一覧、緯度経度など結合
####################################################
df1 = pd.read_csv("~/datasets/ETL/pref_list.csv", encoding="utf8")
df2 = pd.read_csv("~/datasets/ETL/pref_lat_lon.csv", encoding="utf8")
#df1.info()
#df2.info()
df2.rename(columns={'lat': 'pref_latitude' , 'lon': 'pref_logtitude'}, inplace=True)
df3 = pd.merge(df1, df2, on='pref_name')
#df3.info()
df4 = pd.merge(dfSTATION, df3, on='pref_code')
#df4.info()

In [10]:
####################################################
#府県コードに対して採用する観測地点を近距離のものを採用
####################################################
def get_distance(lat1,log1,lat2,log2):
    return geodesic((lat1, log1), (lat2,log2)).km

df4['distance'] = df4[['latitude','logtitude','pref_latitude','pref_logtitude']].apply(
    lambda x: get_distance(x[0],x[1],x[2],x[3]), axis=1)

In [11]:
####################################################
#観測地47都道府県データ
####################################################
df4.to_csv('~/datasets/ETL/poi_df4.csv', encoding='shiftjis')
dfPREF = df4.loc[df4.groupby('pref_code')['distance'].idxmin()]
dfPREF.to_csv('~/datasets/ETL/OUT_PREF_list.csv', encoding='shiftjis')

In [12]:
####################################################
#観測情報取得クラス
####################################################
class AmedasDatabaseScraper(object):
    #----------------------------------------------
    # HTMLテーブルからのカラムでは意味つかめないので補正する
    #----------------------------------------------
    cols_s = ['Day','Pressure1','Pressure2','Precipitation','Precipitation.1hour','Precipitation.1min','Temp.Mean','Temp.Max','Temp.Min','Humidity.Mean','Humidity.Min','WindSpeed.Mean','WindSpeed.Max','WindDirection.Max','MaxInstantaneousWindSpeed','WindDirection.MaxInstantaneous','SunshineTime','Snow','DeepestSnow','WeatherOverview1','WeatherOverview2']
    cols_a = ['Day','Precipitation','Precipitation.1hour','Precipitation.1min','Temp.Mean','Temp.Max','Temp.Min','Humidity.Mean','Humidity.Min','WindSpeed.Mean','WindSpeed.Max','WindDirection.Max','MaxInstantaneousWindSpeed','WindDirection.MaxInstantaneous','MaximumWindDirection','SunshineTime','Snow','DeepestSnow']
    #----------------------------------------------
    # イニシャライザー
    #----------------------------------------------
    def __init__(self, timestep='hourly'):
        self.base_url = 'https://www.data.jma.go.jp/obd/stats/etrn/view/{}_{}1.php?prec_no={}&block_no={}&year={}&month={}&day={}&view=p1'
        assert timestep in ['daily', 'hourly', '10min']
        self.timestep = timestep
        self.prec_no = None
        self.block_no = None
        self.station_type = None
    #----------------------------------------------
    # 初期処理
    #----------------------------------------------
    def initialize(self, prec_no, block_no, station_type, timestep=None):
        self.prec_no = prec_no
        self.block_no = block_no
        self.station_type = station_type
        if timestep is not None:
            assert timestep in ['daily', 'hourly', '10min']
            self.timestep = timestep
    #----------------------------------------------
    # 観測データ取得
    #----------------------------------------------
    def get_data(self, year, month, day):
        assert self.prec_no is not None
        assert self.block_no is not None
        assert self.station_type is not None
        url = self.base_url.format(self.timestep, self.station_type, self.prec_no, self.block_no, year, month, day)
        df = pd.read_html(url)[0]
        cols = [i[-1] for i in df.columns.to_list()]
        if self.station_type == 's':
            df.columns = self.cols_s
        if self.station_type == 'a':
            df.columns = self.cols_a    
        return df

In [13]:
####################################################
#月ごとのループ処理
####################################################
def month_span(start_date, end_date):
    """start_date、end_dateの期間に含まれる月毎のdatetimeオブジェクトを返すジェネレータ
    """
    yield start_date
    while(start_date.year != end_date.year or
          start_date.month != end_date.month):
        start_date = start_date + relativedelta(months=1)
        yield start_date

In [14]:
####################################################
#解析対象を指定し観測情報を生成する
####################################################
today = datetime.today()
last_month = today + relativedelta(months= -1)
before_month = last_month + relativedelta(months= -36)

#print(last_month)
#print(before_month)
#scraper = AmedasDatabaseScraper('daily')
#scraper.initialize(55, 47607, "s")  # 先程取得したデータ
#scraper.initialize(43, "0363", "a")  # 先程取得したデータ
#df = scraper.get_data(2021, 6, 1)  # 取得したい年月日

####################################################
# 解析情報より欠損値など補正、集計値を生成する
####################################################
def dum(prec_no,block_no,amedas_type,pref_name,year,month):
    scraper = AmedasDatabaseScraper('daily')
    #print(f'{prec_no:<5} {block_no:<7} {amedas_type:<2} {pref_name:<10}')
    scraper.initialize(prec_no, block_no, amedas_type)
    df = scraper.get_data(year, month, 1)
    #print(df)
    #値 ) 統計を行う対象資料が許容範囲で欠けている（準正常値）
    #値 ] 統計を行う対象資料が許容範囲を超えて欠けている（資料不足値）
    df['Precipitation'].replace('[\]\)]','', regex=True, inplace=True) 
    df['Precipitation'] = pd.to_numeric(df['Precipitation'], errors="coerce")
    df['Temp.Max'].replace('[\]\)]','', regex=True, inplace=True) 
    df['Temp.Max'] = pd.to_numeric(df['Temp.Max'], errors="coerce")
    df['Humidity.Mean'].replace('[\]\)]','', regex=True, inplace=True) 
    df['Humidity.Mean'] = pd.to_numeric(df['Humidity.Mean'], errors="coerce")
    df['SunshineTime'].replace('[\]\)]','', regex=True, inplace=True) 
    df['SunshineTime'] = pd.to_numeric(df['SunshineTime'], errors="coerce")
    df['Snow'].replace('[\]\)]','', regex=True, inplace=True) 
    df['Snow'] = pd.to_numeric(df['Snow'], errors="coerce")
    df.fillna(0)
    return (year, 
            month, 
            df['Day'].count(), 
            df['Precipitation'].sum(), 
            df['Temp.Max'].mean(),
            df['Humidity.Mean'].mean(),
            df['SunshineTime'].mean(),
            df['Snow'].sum(),
           )
####################################################
# 集計カラム
####################################################
cols_dat = ['Year','Month','Days','Sum.Precipitation','Temp.Max','Humidity.Mean','SunshineTime','Snow']

####################################################
# 指定年月分の情報を作成する
####################################################
df_concat = pd.DataFrame()
for target_date in month_span(before_month, last_month):
    print(f'{target_date.year:04}{target_date.month:02}' )
    dfPREF[cols_dat] = dfPREF[['prec_no', 'block_no', 'amedas_type','pref_name']].apply(
        lambda x: dum(
            x[0], x[1], x[2],x[3], 
            target_date.year, 
            target_date.month
        )
        , axis=1
        , result_type='expand'
    )
    df_concat = pd.concat([df_concat,dfPREF])

201909
201910
201911
201912
202001
202002
202003
202004
202005
202006
202007
202008
202009
202010
202011
202012
202101
202102
202103
202104
202105
202106
202107
202108
202109
202110
202111
202112
202201
202202
202203
202204
202205
202206
202207
202208
202209


In [15]:
####################################################
# 生成情報の出力
####################################################
#df_concat[df_concat['pref_code']==16]
df_concat.to_csv('~/datasets/ETL/dfPREF.csv', encoding='shiftjis')

In [16]:

df_concat

,area,station,prec_no,block_no,amedas_type,latitude,logtitude,pref_code,pref_name,pref_latitude,pref_logtitude,distance,Year,Month,Days,Sum.Precipitation,Temp.Max,Humidity.Mean,SunshineTime,Snow
5,石狩地方,札幌,14,47412,s,43.060000,141.328333,1,北海道,43.064359,141.347449,1.630700,2019.0,9.0,30.0,108.5,24.036667,69.966667,6.950000,0.0
23,青森県,青森,31,47575,s,40.821667,140.768333,2,青森県,40.824294,140.740054,2.403417,2019.0,9.0,30.0,44.5,26.136667,71.700000,6.573333,0.0
27,岩手県,盛岡,33,47584,s,39.698333,141.165000,3,岩手県,39.703530,141.152667,1.204876,2019.0,9.0,30.0,57.0,25.753333,78.833333,5.040000,0.0
31,宮城県,仙台,34,47590,s,38.261667,140.896667,4,宮城県,38.268737,140.872183,2.281901,2019.0,9.0,30.0,72.5,26.563333,75.866667,5.156667,0.0
26,秋田県,秋田,32,47582,s,39.716667,140.098333,5,秋田県,39.718175,140.103356,0.462083,2019.0,9.0,30.0,101.5,26.406667,73.000000,6.206667,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,熊本県,熊本,86,47819,s,32.813333,130.706667,43,熊本県,32.790385,130.742345,4.200459,2022.0,9.0,30.0,144.5,31.080000,68.800000,5.333333,0.0
128,大分県,大分,83,47815,s,33.235000,131.618333,44,大分県,33.238200,131.612674,0.635752,2022.0,9.0,30.0,405.5,28.746667,79.133333,3.756667,0.0
141,宮崎県,宮崎,87,47830,s,31.938333,131.413333,45,宮崎県,31.911090,131.423855,3.180547,2022.0,9.0,30.0,610.0,29.410000,84.066667,4.983333,0.0
145,鹿児島県,鹿児島,88,47827,s,31.555000,130.546667,46,鹿児島県,31.560219,130.557906,1.213914,2022.0,9.0,30.0,284.5,31.193333,75.233333,6.083333,0.0
